# u(x,y) = (1-x^2-y^2)/4

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay


class PINN(nn.Module):
    def __init__(self, num_neurons):
        super(PINN, self).__init__()
        self.input_layer = nn.Linear(2, num_neurons)
        # print(f"Input layer size: {self.input_layer.in_features} -> {self.input_layer.out_features}")

        self.hidden_layers = nn.Sequential(
            nn.Tanh(),
            nn.Linear(num_neurons, num_neurons),
            nn.Tanh(),
            nn.Linear(num_neurons, num_neurons)
        )
        # print(f"Hidden layers configuration: {self.hidden_layers}")

        self.output_layer = nn.Linear(num_neurons, 1)
        # print(f"Output layer size: {self.output_layer.in_features} -> {self.output_layer.out_features}")

        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.hidden_layers(x)
        x = self.output_layer(x)
        return x

def create_circle_mesh(radius=1.0, hmax=0.05):
    theta = np.linspace(0, 2*np.pi, int(2*np.pi/hmax))
    points = np.array([radius * np.cos(theta), radius * np.sin(theta)]).T
    tri = Delaunay(points)
    return points, tri

pdeCoeffs = {'c': 0.5, 'm': 0, 'd': 0, 'a': 0, 'f': 1}

points, tri = create_circle_mesh()
boundary_nodes = np.where(np.linalg.norm(points, axis=1) == 1.0)[0]
domain_nodes = np.setdiff1d(np.arange(len(points)), boundary_nodes)

numNeurons = 50
pinn = PINN(numNeurons)

Input layer size: 2 -> 50
Hidden layers configuration: Sequential(
  (0): Tanh()
  (1): Linear(in_features=50, out_features=50, bias=True)
  (2): Tanh()
  (3): Linear(in_features=50, out_features=50, bias=True)
)
Output layer size: 50 -> 1


In [16]:
# Training setup
numEpochs = 1500
miniBatchSize = 2**12
initialLearnRate = 0.01 #초기 학습률
learnRateDecay = 0.001  #학습률 감쇠 계수
loss_fn = nn.MSELoss()
optimizer = optim.Adam(pinn.parameters(), lr=initialLearnRate)

# Training loop
for epoch in range(numEpochs):
    permutation = np.random.permutation(domain_nodes) #도메인 노드의 인덱스를 무작위로 섞음

    for i in range(0, len(domain_nodes), miniBatchSize): #미니 배치 크기만큼 반복함
        indices = permutation[i:i + miniBatchSize] #현재 미니 배치의 인덱스를 가져옴
        batch_nodes = points[domain_nodes][indices] # 미니 배치의 노드를 선택

        if len(batch_nodes) == 0:
            continue
        XY = torch.tensor(batch_nodes, dtype=torch.float32) #미니 배치 노드를 텐서로 변환함

        # Forward pass
        U = pinn(XY)

        # Compute the loss
        Utrue = (1 - XY[:, 0]**2 - XY[:, 1]**2) / 4 #실제값을 계산
        lossData = loss_fn(U, Utrue.unsqueeze(1))

        # Compute gradients
        gradU = torch.autograd.grad(outputs=U.sum(), inputs=XY, create_graph=True)[0] #U에 대한 XY의 그래디언트를 계산
        Laplacian = torch.zeros_like(U) #라플라시안을 0으로 초기화
        for j in range(2): # 각 차원에 대해 반복
            gradU2 = torch.autograd.grad(outputs=(pdeCoeffs['c'] * gradU[:, j]).sum(), inputs=XY, create_graph=True)[0]
            # 각 차원에 대한 2차 그래디언트 계산
            Laplacian += gradU2[:, j].unsqueeze(1)
            # 라플라시안에 2차 그래디언트를 더함

        # Enforce PDE
        res = -pdeCoeffs['f'] - Laplacian + pdeCoeffs['a'] * U # PDE를 적용한 결과를 계산
        lossF = torch.mean(res**2) # PDE손실 계산

        # Enforce boundary conditions
        BC_XY = torch.tensor(points[boundary_nodes], dtype=torch.float32) #경계 조건을 적용할 노드를 텐서로 변환
        actualBC = torch.zeros(BC_XY.shape[0], 1) #실제 경계 조건 값을 0으로 설정
        predictedBC = pinn(BC_XY) #모델을 통해 경계 조건의 예측값을 계산
        lossBC = loss_fn(predictedBC, actualBC) #예측 경계 조건과 실제 경계 조건 사이의 손실을 계산

        # Combine weighted losses
        lambdaPDE = 0.4
        lambdaBC = 0.6
        lambdaData = 0.5
        loss = lambdaPDE * lossF + lambdaBC * lossBC + lambdaData * lossData

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Learning rate decay
    for param_group in optimizer.param_groups:
        param_group['lr'] = initialLearnRate / (1 + learnRateDecay * (epoch + 1))

    # Print the loss for every epoch
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{numEpochs}], Loss: {loss.item():.4f}')

IndexError: index 23 is out of bounds for axis 0 with size 23